In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

import train

In [2]:
dataset_folder = "../../dataset/"

def refresh_data():
    global artificial_data1
    global artificial_data2
    global msft_data
    global oil_data
    global sp500_data
    global bitcoin_data
    artificial_data1 = pd.read_csv(dataset_folder + "Artificial_trend.csv")
    artificial_data2 = pd.read_csv(dataset_folder + "Artificial_trend1.csv")
    msft_data = pd.read_csv(dataset_folder + "microsoft.csv")
    oil_data = pd.read_csv(dataset_folder + "oil.csv")
    oil_data = oil_data[oil_data.DCOILBRENTEU != "."]
    sp500_data = pd.read_csv(dataset_folder + "sp500.csv")
    bitcoin_data = pd.read_csv(dataset_folder + "coin.csv")

artificial_data1_name = "Artificial Data 1"
artificial_data1_column_i = 1
artificial_data1_date = "date"

artificial_data2_name = "Artificial Data 2"
artificial_data2_column_i = 1
artificial_data2_date = "date"

msft_name = "Microsoft Stock prices"
msft_column_i = 4
msft_date = "Date"

oil_name = "Crude oil prices"
oil_column_i = 1
oil_date = "DATE"

sp500_name = "S&P 500 Index"
sp500_column_i = 1
sp500_date = "Date"

bitcoin_name = "Bitcoin prices"
bitcoin_column_i = 7
bitcoin_date = "Date"

refresh_data()

FileNotFoundError: [Errno 2] No such file or directory: '../../../dataset/Artificial_trend.csv'

# Traditional RNN

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size, n_layers, drop_prob=0.2):
        super(RNN, self).__init__()
        
        #Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        #Defining layers
        #RNN, use dropout to avoid overfitting
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        #Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
        #activation function, using RELU to prevent shrinking the gradient
        self.relu = nn.ReLU()
    
    def forward(self, x, h):
        out, h = self.rnn(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self,batch_size):
        #generates the first hidden state of zeros used in the forward pass
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        return hidden

In [ ]:
refresh_data()
network = RNN
model_name = "RNN"

## Artificial Data 1

In [ ]:
train.run(artificial_data1, artificial_data1_column_i, artificial_data1_name, artificial_data1_date, network, model_name)

X_train.shape:  (5842, 5, 1)
y_train.shape:  (5842, 1)
X_test.shape:  (1460, 5, 1)
y_test.shape:  (1460, 1)
Starting Training of RNN model
<built-in method type of Tensor object at 0x00000202D22C9540>


C:\Users\goodu\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/50 Done, Total Training MSE Loss: 0.0550306417752546
Total Testing MSELoss: 0.007278489880263805
Total Time Elapsed: 0.8497930000000053 seconds

<built-in method type of Tensor object at 0x00000202D90E71D0>
Epoch 2/50 Done, Total Training MSE Loss: 0.040673312394953955
Total Testing MSELoss: 0.014760928228497505
Total Time Elapsed: 0.8013402999999926 seconds

<built-in method type of Tensor object at 0x00000202D90EA4A0>
Epoch 3/50 Done, Total Training MSE Loss: 0.043150707419864466
Total Testing MSELoss: 0.02455042116343975
Total Time Elapsed: 0.848704099999992 seconds

<built-in method type of Tensor object at 0x00000202D90EB270>
Epoch 4/50 Done, Total Training MSE Loss: 0.040580663477517744
Total Testing MSELoss: 0.02134319394826889
Total Time Elapsed: 0.827299599999975 seconds

<built-in method type of Tensor object at 0x00000202D90ED4A0>
Epoch 5/50 Done, Total Training MSE Loss: 0.03880705700446545
Total Testing MSELoss: 0.027330685406923294
Total Time Elapsed: 0.8130182000

KeyboardInterrupt: 

## Artificial Data 2

In [ ]:
train.run(artificial_data2, artificial_data2_column_i, artificial_data2_name, artificial_data2_date, network, model_name)

## Microsoft Stock Price

In [ ]:
train.run(msft_data, msft_column_i, msft_name, msft_date, network, model_name)

## Crude oil price

In [ ]:
train.run(oil_data, oil_column_i, oil_name, oil_date, network, model_name)

## S&P 500 Index

In [ ]:
train.run(sp500_data, sp500_column_i, sp500_name, sp500_date, network, model_name)

## Bitcoin price

In [ ]:
train.run(bitcoin_data, bitcoin_column_i, bitcoin_name, bitcoin_date, network, model_name)

# Bidirectional RNN

In [ ]:
class biRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(biRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.biRNN = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim*2, output_dim)

    def forward(self, x, h):
        # h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).requires_grad_()
        # c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, h = self.biRNN(x, h)
        out = self.fc(out[:, -1, :]) 
        return out, h

    def init_hidden(self, batch_size):
        #generates the first hidden state of zeros used in the forward pass
        weight = next(self.parameters()).data
        hidden = weight.new(self.num_layers*2, batch_size, self.hidden_dim).zero_()
        return hidden

In [ ]:
refresh_data()
network = biRNN
model_name = "Bidirectional RNN"

## Artificial Data 1

In [ ]:
train.run(artificial_data1, artificial_data1_column_i, artificial_data1_name, artificial_data1_date, network, model_name)

## Artificial Data 2

In [ ]:
train.run(artificial_data2, artificial_data2_column_i, artificial_data2_name, artificial_data2_date, network, model_name)

## Microsoft Stock Price

In [ ]:
train.run(msft_data, msft_column_i, msft_name, msft_date, network, model_name)

## Crude oil price

In [ ]:
train.run(oil_data, oil_column_i, oil_name, oil_date, network, model_name)

## S&P 500 Index

In [ ]:
train.run(sp500_data, sp500_column_i, sp500_name, sp500_date, network, model_name)

## Bitcoin price

In [ ]:
train.run(bitcoin_data, bitcoin_column_i, bitcoin_name, bitcoin_date, network, model_name)

# Gated Recurrent Unit (GRU)

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.1):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(out[:,-1])
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        return hidden

In [ ]:
refresh_data()
network = GRU
model_name = "GRU"

## Artificial Data 1

In [ ]:
train.run(artificial_data1, artificial_data1_column_i, artificial_data1_name, artificial_data1_date, network, model_name)

## Artificial Data 2

In [ ]:
train.run(artificial_data2, artificial_data2_column_i, artificial_data2_name, artificial_data2_date, network, model_name)

## Microsoft Stock Price

In [ ]:
train.run(msft_data, msft_column_i, msft_name, msft_date, network, model_name)

## Crude oil price

In [ ]:
train.run(oil_data, oil_column_i, oil_name, oil_date, network, model_name)

## S&P 500 Index

In [ ]:
train.run(sp500_data, sp500_column_i, sp500_name, sp500_date, network, model_name)

## Bitcoin price

In [ ]:
train.run(bitcoin_data, bitcoin_column_i, bitcoin_name, bitcoin_date, network, model_name)

# Long-Short Term Memory

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x, ph):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out, hn

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(0, 0, 0).zero_()
        return hidden

In [ ]:
refresh_data()
network = LSTM
model_name = "LSTM"

## Artificial Data 1

In [ ]:
train.run(artificial_data1, artificial_data1_column_i, artificial_data1_name, artificial_data1_date, network, model_name)

## Artificial Data 2

In [ ]:
train.run(artificial_data2, artificial_data2_column_i, artificial_data2_name, artificial_data2_date, network, model_name)

## Microsoft Stock Price

In [ ]:
train.run(msft_data, msft_column_i, msft_name, msft_date, network, model_name)

## Crude oil price

In [ ]:
train.run(oil_data, oil_column_i, oil_name, oil_date, network, model_name)

## S&P 500 Index

In [ ]:
train.run(sp500_data, sp500_column_i, sp500_name, sp500_date, network, model_name)

## Bitcoin price

In [ ]:
train.run(bitcoin_data, bitcoin_column_i, bitcoin_name, bitcoin_date, network, model_name)